# Testing Individual components of the FV HE scheme

In [1]:
from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.encrypter import Encrypter
from syft.frameworks.torch.he.fv.decrypter import Decrypter
from syft.frameworks.torch.he.fv.integer_encoder import IntegerEncoder
from syft.frameworks.torch.he.fv.modulus import SeqLevelType

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/Users/ravikantsingh/Desktop/PySyft/venv/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'



## Generates coeff_modulus using Polymodulus degree and bit-length parameters.
Generates random integer of the given bit length for using as coeff_modulus

In [2]:
params = EncryptionParams()
cm = CoeffModulus()
params.coeff_modulus = cm.create(4096, [30, 50, 60])
print("coeff_modulus: ", params.coeff_modulus)

coeff_modulus:  [1073692673, 1125899906826241, 1152921504606830593]


## Use standard values of coeff_modulus for the given Poly Modulus degree and security_level.
Need some refractoring and integration of code.

In [3]:
params_standard = EncryptionParams()
cm_standard = CoeffModulus()
for i in SeqLevelType:
    for j in [1024, 4096, 32768]:           # ... [1024, 2048, 4096, 8192, 16384, 32768]
        params_standard.coeff_modulus = cm_standard.bfv_default(j, i)
        print(f"coeff_modulus for {i} and poly_modulus = {j}", params_standard.coeff_modulus)
    print('\n')

coeff_modulus for SeqLevelType.TC128 and poly_modulus = 1024 [132120577]
coeff_modulus for SeqLevelType.TC128 and poly_modulus = 4096 [68719403009, 68719230977, 137438822401]
coeff_modulus for SeqLevelType.TC128 and poly_modulus = 32768 [36028797017456641, 36028797014704129, 36028797014573057, 36028797014376449, 36028797013327873, 36028797013000193, 36028797012606977, 36028797010444289, 36028797009985537, 36028797005856769, 36028797005529089, 36028797005135873, 36028797003694081, 36028797003563009, 36028797001138177, 72057594037338113]


coeff_modulus for SeqLevelType.TC192 and poly_modulus = 1024 [520193]
coeff_modulus for SeqLevelType.TC192 and poly_modulus = 4096 [33538049, 33349633, 33292289]
coeff_modulus for SeqLevelType.TC192 and poly_modulus = 32768 [18014398506729473, 18014398505943041, 18014398499848193, 18014398498799617, 18014398498275329, 36028797017456641, 36028797014704129, 36028797014573057, 36028797014376449, 36028797013327873, 36028797013000193]


coeff_modulus for Se

## Keygeneration

In [14]:
params = EncryptionParams(4096, CoeffModulus().create(4096, [30, 50, 60]) , 64 )
ctx = Context(params)
keygenerator = KeyGenerator(ctx)
sk, pk = keygenerator.keygen()

In [15]:
print('secret key values : ', sk.data)
print(sk)

secret key values :  tensor([33538048,        1, 33538048,  ...,        1, 33292288,        1])


In [16]:
print(pk.data[1])

tensor([ 7242040, 11722677, 14585196,  ..., 17361490, 11126279, 12547784])


In [17]:
print('public key values : ', pk.data)

public key values :[[tensor(7242036), tensor(21815368), tensor(14585195), tensor(1), tensor(18856655), tensor(33538047), tensor(5321916), tensor(22669512), tensor(12985903), tensor(18764743), tensor(30725602), tensor(33538047), tensor(32606739), tensor(30493035), tensor(29308716), tensor(10103978), tensor(3), tensor(30392809), tensor(0), tensor(33206991), tensor(25678402), tensor(2402739), tensor(22551289), tensor(15337287), tensor(33538041), tensor(31249221), tensor(17462153), tensor(11049226), tensor(13750275), tensor(8405052), tensor(0), tensor(14198451), tensor(25693566), tensor(33538047), tensor(33538047), tensor(4), tensor(13732123), tensor(33538047), tensor(7842257), tensor(10756732), tensor(3323454), tensor(11800669), tensor(13182023), tensor(30525514), tensor(33538046), tensor(0), tensor(0), tensor(33538047), tensor(30727196), tensor(12163393), tensor(5), tensor(26343045), tensor(3), tensor(33334652), tensor(27428280), tensor(25136455), tensor(2), tensor(15757783), tensor(2134

## Integer Encoder
Encodes Integer values to Plaintext object

In [8]:
int_encoder = IntegerEncoder(ctx)
plaintext = int_encoder.encode(100)
print(plaintext)
print('plaintext data',plaintext.data)

plaintext data [0, 0, 1, 0, 0, 1, 1]


### Decodes back to Integer

In [9]:
print(int_encoder.decode(plaintext))

100


## Encrypter
Encrypt Plaintext to ciphertext using public_key

In [10]:
encrypter = Encrypter(ctx, pk)
ciphertext = encrypter.encrypt(plaintext)
print(ciphertext)
print('ciphertext data :', ciphertext.data)

plain_0 <class 'list'>
ciphertext data :[[tensor(865179422), tensor(1), tensor(201450958), tensor(440349835), tensor(652422109), tensor(0), tensor(1017871120), tensor(0), tensor(750613808), tensor(705337980), tensor(1073692671), tensor(834060776), tensor(957954932), tensor(527035743), tensor(160358432), tensor(166637894), tensor(465227368), tensor(3), tensor(1073692668), tensor(1073692672), tensor(1073692670), tensor(553025930), tensor(2), tensor(533326352), tensor(109282449), tensor(216931174), tensor(789921846), tensor(3), tensor(771390555), tensor(3), tensor(4), tensor(202577677), tensor(1073692670), tensor(1), tensor(1073692669), tensor(610942912), tensor(1073692670), tensor(1073692669), tensor(3), tensor(343418634), tensor(27353968), tensor(1053602860), tensor(1), tensor(733775850), tensor(0), tensor(1073692672), tensor(175619671), tensor(358092865), tensor(1073692665), tensor(397488241), tensor(83418940), tensor(397139143), tensor(712851073), tensor(3), tensor(1073692670), tensor

In [11]:
# TODO Need some changes.

# encrypter_symmetric = Encrypter(ctx, sk)
# ciphertext_symmetric = encrypter_symmetric.encrypt(plaintext)
# print(ciphertext_symmetric)

Encrypt Plaintext to ciphertext using secret_key

## Decrypter
Decrypt Ciphertext to Plaintext using secret_key

In [12]:
# Waiting for SEAL v3.5

# decrypter = Decrypter(ctx, sk)
# result = decrypter.decrypt(ciphertext)
# print("len : ",len(result.data))
# print(result.data)
# print(result)

In [13]:
# print(int_encoder.decode(result))

False